In [5]:
import dotenv
import os
dotenv.load_dotenv()


True

In [6]:
KEY=os.getenv("API_KEY")

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=KEY)
print(llm.invoke("top 10 country in asia in terms of GDP").content)

1. China
2. Japan
3. India
4. Indonesia
5. South Korea
6. Thailand
7. Turkey
8. Saudi Arabia
9. Iran
10. Pakistan


In [10]:
import streamlit as st
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
import PyPDF2


In [11]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [12]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker.Given the about text, it is your job to\
create a quize of {number} multiple choice questions fro {subject} students in {tone} tone.
Make sure the questions are not repated and check all questions to be conforming the text as well.
Make sure to Format your Response like RESPONSE_JSON below use it as guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""


In [13]:
quiz_generator_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [14]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generator_prompt,
    output_key="quiz",
    verbose=True
)

In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [16]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [17]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key="review",
    verbose=True
)


In [18]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"]
)

In [20]:
import json
json.dumps(RESPONSE_JSON)


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5 
SUBJECT="AI and ML"
TONE="simple"

In [22]:
TEXT=""" Artificial intelligence (AI) and machine learning are often used interchangeably, but machine learning is a subset of the broader category of AI.

Put in context, artificial intelligence refers to the general ability of computers to emulate human thought and perform tasks in real-world environments, while machine learning refers to the technologies and algorithms that enable systems to identify patterns, make decisions, and improve themselves through experience and data. 

Computer programmers and software developers enable computers to analyze data and solve problems — essentially, they create artificial intelligence systems — by applying tools such as:

machine learning
deep learning
neural networks
computer vision
natural language processing
Below is a breakdown of the differences between artificial intelligence and machine learning as well as how they are being applied in organizations large and small today.

What Is Artificial Intelligence?
Artificial Intelligence is the field of developing computers and robots that are capable of behaving in ways that both mimic and go beyond human capabilities. AI-enabled programs can analyze and contextualize data to provide information or automatically trigger actions without human interference.

Today, artificial intelligence is at the heart of many technologies we use, including smart devices and voice assistants such as Siri on Apple devices. Companies are incorporating techniques such as natural language processing and computer vision — the ability for computers to use human language and interpret images ­— to automate tasks, accelerate decision making, and enable customer conversations with chatbots.

What Is Machine Learning?
Machine learning is a pathway to artificial intelligence. This subcategory of AI uses algorithms to automatically learn insights and recognize patterns from data, applying that learning to make increasingly better decisions.

By studying and experimenting with machine learning, programmers test the limits of how much they can improve the perception, cognition, and action of a computer system.

Deep learning, an advanced method of machine learning, goes a step further. Deep learning models use large neural networks — networks that function like a human brain to logically analyze data — to learn complex patterns and make predictions independent of human input."""

In [23]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

d:\MCQ_Generator\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:

Text: Artificial intelligence (AI) and machine learning are often used interchangeably, but machine learning is a subset of the broader category of AI.

Put in context, artificial intelligence refers to the general ability of computers to emulate human thought and perform tasks in real-world environments, while machine learning refers to the technologies and algorithms that enable systems to identify patterns, make decisions, and improve themselves through experience and data. 

Computer programmers and software developers enable computers to analyze data and solve problems — essentially, they create artificial intelligence systems — by applying tools such as:

machine learning
deep learning
neural networks
computer vision
natural language processing
Below is a breakdown of the differences between artificial intelligence and machine learning as well as how they are being applied in organizations large and small today.

What I

In [29]:
quiz=response["quiz"].split("\n")[1]

In [39]:
quiz = json.loads(quiz)
quiz

{'1': {'mcq': 'Which of the following is NOT a tool used to create artificial intelligence systems?',
  'options': {'a': 'Machine learning',
   'b': 'Deep learning',
   'c': 'Computer vision',
   'd': 'Cloud computing'},
  'correct': 'd'},
 '2': {'mcq': 'What is the primary difference between artificial intelligence and machine learning?',
  'options': {'a': 'Artificial intelligence is a broader category that includes machine learning',
   'b': 'Machine learning is a more advanced form of artificial intelligence',
   'c': 'Artificial intelligence focuses on human-like behavior, while machine learning focuses on data analysis',
   'd': 'There is no difference between artificial intelligence and machine learning'},
  'correct': 'a'},
 '3': {'mcq': 'Which type of machine learning uses large neural networks to learn complex patterns?',
  'options': {'a': 'Supervised learning',
   'b': 'Unsupervised learning',
   'c': 'Reinforcement learning',
   'd': 'Deep learning'},
  'correct': 'd'},
 '

In [32]:
response.get("quiz")

'### RESPONSE_JSON\n{"1": {"mcq": "Which of the following is NOT a tool used to create artificial intelligence systems?", "options": {"a": "Machine learning", "b": "Deep learning", "c": "Computer vision", "d": "Cloud computing"}, "correct": "d"}, "2": {"mcq": "What is the primary difference between artificial intelligence and machine learning?", "options": {"a": "Artificial intelligence is a broader category that includes machine learning", "b": "Machine learning is a more advanced form of artificial intelligence", "c": "Artificial intelligence focuses on human-like behavior, while machine learning focuses on data analysis", "d": "There is no difference between artificial intelligence and machine learning"}, "correct": "a"}, "3": {"mcq": "Which type of machine learning uses large neural networks to learn complex patterns?", "options": {"a": "Supervised learning", "b": "Unsupervised learning", "c": "Reinforcement learning", "d": "Deep learning"}, "correct": "d"}, "4": {"mcq": "What is t

In [30]:
response.get("review")


"**Complexity Analysis:**\n\nThe quiz questions are clear and concise, with options that cover the key concepts of AI and ML. The questions are appropriate for students with a basic understanding of these fields.\n\n**Updated Quiz Questions:**\n\nThe quiz is well-structured and covers the essential concepts of AI and ML. No changes are necessary.\n\n**Tone:**\n\nThe tone of the quiz is professional and objective, which is appropriate for an assessment of students' knowledge."

In [40]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [45]:
import pandas as pd
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,Which of the following is NOT a tool used to c...,a: Machine learning | b: Deep learning | c: Co...,d
1,What is the primary difference between artific...,a: Artificial intelligence is a broader catego...,a
2,Which type of machine learning uses large neur...,a: Supervised learning | b: Unsupervised learn...,d
3,What is the goal of machine learning algorithms?,a: To identify patterns and make decisions | b...,a
4,Which of the following is an example of an app...,a: Smart home devices | b: Medical diagnosis |...,d
